In [2]:
import pandas as pd

In [22]:
github_url = "https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/04-monitoring/data/results-gpt4o-mini.csv"
url = f'{github_url}?raw=1'
df = pd.read_csv(url)

In [23]:
df = df.iloc[:300]

# Q1

In [5]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer("multi-qa-mpnet-base-dot-v1")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

You try to use a model that was created with version 3.0.0.dev0, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





README.md:   0%|          | 0.00/8.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
list(df.columns)

['answer_llm', 'answer_orig', 'document', 'question', 'course']

In [8]:
answer_llm = df.iloc[0].answer_llm
embedding_model.encode(answer_llm)[0]

-0.42244688

# Q2

In [24]:
def compute_similarity(record):
    answer_orig = record['answer_orig']
    answer_llm = record['answer_llm']
    
    v_llm = embedding_model.encode(answer_llm)
    v_orig = embedding_model.encode(answer_orig)
    
    return v_llm.dot(v_orig)

In [25]:
df_rec = df.to_dict(orient='records')

In [26]:
from tqdm.auto import tqdm
evaluations = []

for record in tqdm(df_rec):
    sim = compute_similarity(record)
    evaluations.append(sim)

  0%|          | 0/300 [00:00<?, ?it/s]

In [27]:
df["dot"] = evaluations
df["dot"].describe()

count    300.000000
mean      27.495996
std        6.384744
min        4.547924
25%       24.307850
50%       28.336860
75%       31.674304
max       39.476013
Name: dot, dtype: float64

# Q3

In [28]:
import numpy as np

In [29]:
def normalize_vector(v):
    norm = np.sqrt((v * v).sum())
    return v / norm

In [33]:
def compute_cosine_similarity(record):
    answer_orig = record['answer_orig']
    answer_llm = record['answer_llm']
    
    v_llm = embedding_model.encode(answer_llm)
    v_llm = normalize_vector(v_llm)
    v_orig = embedding_model.encode(answer_orig)
    v_orig = normalize_vector(v_orig)
    
    return v_llm.dot(v_orig)

In [34]:
from tqdm.auto import tqdm
evaluations = []

for record in tqdm(df_rec):
    sim = compute_cosine_similarity(record)
    evaluations.append(sim)

  0%|          | 0/300 [00:00<?, ?it/s]

In [35]:
df["cosine"] = evaluations
df["cosine"].describe()

count    300.000000
mean       0.728392
std        0.157755
min        0.125357
25%        0.651273
50%        0.763761
75%        0.836235
max        0.958796
Name: cosine, dtype: float64

# Q4

In [36]:
!pip install rouge

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [40]:
assert df_rec[10]["document"] == "5170565b"

In [41]:
r = df_rec[10]

In [42]:
from rouge import Rouge
rouge_scorer = Rouge()

scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]

In [43]:
scores

{'rouge-1': {'r': 0.45454545454545453,
  'p': 0.45454545454545453,
  'f': 0.45454544954545456},
 'rouge-2': {'r': 0.21621621621621623,
  'p': 0.21621621621621623,
  'f': 0.21621621121621637},
 'rouge-l': {'r': 0.3939393939393939,
  'p': 0.3939393939393939,
  'f': 0.393939388939394}}

In [44]:
scores['rouge-1']['f']

0.45454544954545456

# Q5

In [45]:
rouge_1 = scores['rouge-1']['f']
rouge_2 = scores['rouge-2']['f']
rouge_l = scores['rouge-l']['f']
rouge_avg = (rouge_1 + rouge_2 + rouge_l) / 3

In [46]:
rouge_avg

0.35490034990035496

# Q6

In [50]:
rouge_scorer = Rouge()

def compute_rouge_score(record):
    scores = rouge_scorer.get_scores(record['answer_llm'], record['answer_orig'])[0]
    rouge_1 = scores['rouge-1']['f']
    rouge_2 = scores['rouge-2']['f']
    rouge_l = scores['rouge-l']['f']
    return rouge_1, rouge_2, rouge_l

In [51]:
from tqdm.auto import tqdm
scores_rouge_1 = []
scores_rouge_2 = []
scores_rouge_l = []

for record in tqdm(df_rec):
    rouge_1, rouge_2, rouge_l = compute_rouge_score(record)
    scores_rouge_1.append(rouge_1)
    scores_rouge_2.append(rouge_2)
    scores_rouge_l.append(rouge_l)

  0%|          | 0/300 [00:00<?, ?it/s]

In [52]:
df["rouge_1"] = scores_rouge_1
df["rouge_2"] = scores_rouge_2
df["rouge_l"] = scores_rouge_l
df["rouge_2"].describe()

count    300.000000
mean       0.206965
std        0.153550
min        0.000000
25%        0.097809
50%        0.178671
75%        0.286181
max        0.739130
Name: rouge_2, dtype: float64